In [ ]:
# Test Cell
# When the notebook is executed by the widget, this cell is ignored.

import pandas as pd
import sys
import os
import base64
import io

additional_parameters = "{\"question\":\"What is the main organ in the body that is affected by each of the following diseases? [dashboard sample healthcare widget TOP 10 DIAGNOSIS column diagnosis]\n\""+ \
",\"widgetId\":\"63ac28e77a19050033efd92d\",\"model_name\":\"Sample Healthcare\"" + \
",\"table_name\":\"Temp\",\"cookie\":\"XXX\"}"


df_input = pd.DataFrame()
csv_input =  "QmVmb3JlXG5DT08="
print('size of csv_input is:' + str(sys.getsizeof(csv_input)) + ' bytes')
bytes = base64.b64decode(csv_input).decode('unicode_escape')
df_input = pd.read_csv(io.StringIO(bytes), na_values='!#NULL#!')

# for local develop
# Insert IP of Sisense instance
# os.environ['API_GATEWAY_EXTERNAL_SERVICE_HOST'] = "X.X.X.X"
# # Insert Port of Sisense instance
# os.environ['API_GATEWAY_EXTERNAL_SERVICE_PORT'] = "30845"
# # Insert Cookie received by Sisense REST API - '/authentication/tokens/api'
# os.environ['Cookie'] = "XXX"

# Getting Started

Welcome to the AI-model (GPT) Data Search Notebook.  

This notebook shows how to augment data with GPT model. Using this notebook it is possible to:
1. Augment sisense data with GPT model, without using existing data in Sisense.
1. Augment sisense data with GPT model, with using existing data in Sisense.

Data extraction from Sisense can be done in 2 ways:
1. Specify the table and column name in the prompt question ([country.country])
1. Specify the table and column name in the prompt question ([table country column country])
1. Spacify the dashboard, widget and column name in the prompt question ([dashboard sample healthcare widget TOP 10 DIAGNOSIS column diagnosis])

The extraction info from sisense column should be described within square bracket ("Give me the language in the following countries [country.country]")
    
---

#### Jupyter Notebooks
If you are not familiar with Jupyter Notebooks, we suggest reading about it [here](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/What%20is%20the%20Jupyter%20Notebook.html).

Get to know the Basics of Jupyter Notebooks, including how to add a New Notebook manually to the Jupyter Server [here](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html).

---

This Tutorial shows the following basic principles:
1. Examples for API parameters
1. How to parse the prompt question and extract sisense data info.
1. How to create requests to GPT model in a batch mode
1. How to parse GPT response
1. How to insert the response in to a table in a live model
1. how to return the response in html format

---
## Notebook Flow

![./BloxAIFlows-DataSearch.png](./BloxAIFlows-DataSearch.png)

---

## Content
  
1. [Test Cell](#Test-Cell)
2. [Imports](#Imports)
3. [Additional Parameters](#Additional-Parameters)
4. [Initialize Global Variables](#Initialize-Global-Variables)
5. [Search Fields In Sisense](#Search-Fields-In-Sisense)
6. [Ask AI Model Questions](#Ask-AI-Model-Questions)
7. [Identify The Data Types Of The Generated Data](#Identify-The-Data-Types-Of-The-Generated-Data)
8. [Create Table With Generated Data](#Create-Table-With-Generated-Data)
9. [Publish The Model](#Publish-The-Model)
10. [Return HTML Result](#Return-HTML-Result)


<a id='How_to_run_the_notebook'></a>
## How to run the notebook
There are two options to execute this notebook, the first, which is the default one is via Jupyter server in Sisense instance. We refer this option as the "Remote" option.

The second one is "local" execution via local IDE.

The default behavior for the code in this notebook is to run in remote.
To run it locally, some code changes are required. To apply the changes, you can uncomment the code under the title - "for local develop".

The changes are in the following cells:
1. [Test Cell](#Test-Cell)
    Insert the IP of Sisense instance to environment variable - "API_GATEWAY_EXTERNAL_SERVICE_HOST"
    Insert the Port of Sisense instance to environment variable - "API_GATEWAY_EXTERNAL_SERVICE_PORT"
    Insert authentication token for Sisense user to environment variable - "Cookie" (How to get Sisense Authentication token is detailed explained [here](https://gitlab.sisense.com/SisenseTeam/DataSciense/bloxai/-/blob/master/README.md))
2. [Imports](#Imports)
    Change the import statements to local repository structure
3. [Additional Parameters](#Additional-Parameters)
    Change to extract authentication token from environment variable - "Cookie"

# Test Cell

The First notebook cell is a Test Cell. 

### What is a Test Cell ?
The additional parameters in the Test Cell are sample values. Its purpose is to mimic the additional parameters passed to the notebook from the Sisense Data Model build process. By having this test cell, the notebook can be run independently of the build process for debugging or editing purposes.   
When the notebook is executed from the Sisense build process, the test cell is skipped and the additional parameters are passed to the notebook downstream as a JSON string.  
You can change the Test cell location by changing its value in the notebook manifest. The default value is always the first notebook cell (cell 0).   

> "cellsDisable": [0]


# Imports

In [ ]:
import pandas as pd
import json
import re

# for remote develop
from AIIntegration import AIIntegration
from AIUtils import AIUtils
from InferenceQuestionType import QType

# for local develop
# from custom_code_notebooks.utils.AIIntegration import AIIntegration
# from custom_code_notebooks.utils.AIUtils import AIUtils
# from custom_code_notebooks.utils.InferenceQuestionType import QType

# Additional Parameters

Load the additional parameters passed from the query process or those initialized in the Test Cell above.

In this example multiple parameters are passed to the notebook:
1. question: a prompt question from the user ("Please give me the company industry of the following companies: [Churn75K.NAME])
1. widgetId: the widget id of the widget requesting the custom code notebook excecution
1. model_name: the model name of the dashboard (and widget) requesting the custom code notebook excecution 
1. table_name: the table name that will be genereated with the external data
1. cookie: sisense user authentication token

In [ ]:
print (additional_parameters)
add_param = json.loads(additional_parameters,strict=False)

orig_question = add_param['question'] 
model_name = add_param['model_name']
table_name = add_param['table_name']
inputWidgetId = add_param['widgetId']
# for remote develop
cookie = add_param['cookie']
# for local develop
# cookie = os.getenv("Cookie")

# Initialize Global Variables

the next cell will initialize the following variables:

**abort** - boolean value, indicate if the process got an exception or unexpected behavior and should stop execution.

**df_result** - dataFrame that will contain the notebook's output, the default value will indicate that 'Something went wrong'.

**model** - data model object- corresponding to model_name parameter.

**oai** - a connection to ai integration library

**utils** - a connection to utils library, contain a logger array that used as time tracking for performance logging. also contain a logger array that used as query tracker for both AI model and Jaql queries.


**num_item_per_split** - number of items in a batch mode, to query with sisense data


In [ ]:
abort = False
utils = AIUtils(inputWidgetId, model_name, add_param, cookie)
utils.add_time('Start')

df_result = pd.DataFrame(data=['Something went wrong'], columns=['Error'])

model = utils.get_model(model_name)
datamodelId = model['oid']

datasetId = model['datasets'][0]['oid']


utils.add_time('before import')
oai = AIIntegration(utils)
utils.add_time('After import')

num_item_per_split = 8

utils.write_text_to_log_updates('<H1>AI start searching</H1>')

# Search Fields In Sisense
Query data from Sisense if asked in the prompt question [BigOpportunity.ACCOUNT_NAME__C]
## Steps:
1. Identify question type from prompt
2. Identify entities in the prompt
3. Get relevant dashboard/widget candidates from Sisense
4. Identify the candidates that best fit to prompt entities
5. Query data from Sisense

In [ ]:
question = orig_question + "\noutput format as JSON.\n"
prompt = question + '\n'
prompt_vec = [prompt]
# extract text from square brackets
get_columns_from_model = re.findall(r'\[.*?\]',question)


if len(get_columns_from_model) > 0:
    requested_data = get_columns_from_model[0]
    print("requested_column " + requested_data)
    options = [QType.TypeColumnTable,QType.TypeDashboardWidgetColumn]
    [jaql, info, question_type_object] = oai.get_elements(model_name,requested_data,options)
    
    utils.write_log_updates()

    if info[0].startswith('Error'):
        df_result = oai.handle_error_in_element_extraction(info[0])
    abort, data = oai.get_data_from_jaql(jaql, model_name) 
    if not abort:
        column_data = oai.get_column_data(data, jaql, question_type_object) 
        prompt_vec = oai.generate_query_to_ai_based_on_column_data(orig_question, requested_data, column_data, num_item_per_split) 
        
    utils.write_log_updates()

# Ask AI Model Questions
Since Sisense data can be large, we split the questions to AI model and ask it in batch mode, where each quaesion contain subset of the data

In [ ]:
if not abort:
    ai_responses = oai.ask_ai_in_batch_mode(prompt_vec)
    abort, df_result = oai.combine_responses_in_batch_mode(prompt_vec, ai_responses, num_item_per_split)
    
utils.write_log_updates()   
df_result.transpose()

# Identify The Data Types Of The Generated Data


In [ ]:
if not abort:
    types = oai.get_data_types_for_response(df_result)  

# Create Table With Generated Data
## steps:
1. Building the import query
2. Delete table with same name as output table (if exist)
3. Create output table with sql import query

In [ ]:
if not abort:
    sql_import_query = utils.get_import_query(df_result, model, types)
    utils.delete_table(model, table_name)
    df_result_keys = df_result.keys()
    utils.create_table_with_query(model, sql_import_query, table_name, types, df_result_keys)

# Publish The Model


In [ ]:
if not abort:
    utils.publish_model(model_name, datamodelId)

# Return HTML Result
## steps:
1. Create a jaql widget based on output table
2. create HTML result

In [ ]:
widget_pararm_str = ""
if not abort:
    widget_pararm_str = utils.get_output_widget_jaql(table_name, types, df_result_keys)
df_result = utils.generate_html_result(widget_pararm_str, df_result)

In [ ]:
df_result

In [ ]:
# from IPython.display import display, HTML
# display(HTML("<html>" + df_result['output'][0]+"</html>"))